In [ ]:
%%cu

#include <stdio.h>

#define N 27
#define BLOCK_SIZE 5

__global__ void izracunajB(float* A, float* B)
{
    __shared__ float sharedA[BLOCK_SIZE + 2];

    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < N)
    {
        if (threadIdx.x<BLOCK_SIZE){
        sharedA[threadIdx.x] = A[tid];}
        if (threadIdx.x == 0 )
        {
            sharedA[BLOCK_SIZE] = A[tid+BLOCK_SIZE];
        }
        if (threadIdx.x == 1 )
        {
            sharedA[BLOCK_SIZE +1] = A[tid+BLOCK_SIZE];
        }
    }
    __syncthreads();

    if (tid < N - 2)
    {

        B[tid] =  (sharedA[threadIdx.x] * sharedA[threadIdx.x + 1] * sharedA[threadIdx.x + 2]) / (sharedA[threadIdx.x] + sharedA[threadIdx.x + 1] + sharedA[threadIdx.x + 2]);
    }
}

int main()
{
    float A[27] = {1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22, 23, 24,25,26,27};
    float B[27 - 2];

    float* d_A;
    float* d_B;
    cudaMalloc((void**)&d_A, N * sizeof(float));
    cudaMalloc((void**)&d_B, (N - 2) * sizeof(float));

    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    int numBlocks = (N - 2 + BLOCK_SIZE - 1) / BLOCK_SIZE;
    izracunajB<<<numBlocks, BLOCK_SIZE>>>(d_A, d_B);

    cudaMemcpy(B, d_B, (N - 2) * sizeof(float), cudaMemcpyDeviceToHost);

    printf("B = ");
    for (int i = 0; i < N - 2; i++)
    {
        printf("%f ", B[i]);
    }
    printf("\n");
    cudaFree(d_A);
    cudaFree(d_B);

    return 0;
}


B = 1.000000 2.666667 5.000000 8.000000 11.666667 16.000000 21.000000 26.666666 33.000000 40.000000 47.666668 56.000000 65.000000 74.666664 85.000000 96.000000 107.666664 120.000000 133.000000 146.666672 161.000000 176.000000 191.666672 208.000000 225.000000 



In [ ]:
%%cu

#include <stdio.h>

#define N 27
#define BLOCK_SIZE 4

__global__ void izracunajB(float* A, float* B)
{
    __shared__ float sharedA[BLOCK_SIZE + 2];

    int tid = threadIdx.x;
    int indeks = blockIdx.x * blockDim.x + threadIdx.x;

    // Load data into shared memory
    if (indeks < N)
    {
        sharedA[tid] = A[indeks];
        if (tid == BLOCK_SIZE-2)
        {

            sharedA[BLOCK_SIZE] = A[indeks + 2];
        }
        if (tid == BLOCK_SIZE-1)
        {

            sharedA[BLOCK_SIZE+1] = A[indeks + 2];
        }
    }
    __syncthreads();

    if (indeks < N - 2)
    {

        B[indeks] =  (sharedA[tid] * sharedA[tid + 1] * sharedA[tid + 2]) / (sharedA[tid] + sharedA[tid + 1] + sharedA[tid + 2]);
    }
}

int main()
{
    float A[27] = {1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22, 23, 24,25,26,27};
    float B[27 - 2];

    float* d_A;
    float* d_B;
    cudaMalloc((void**)&d_A, N * sizeof(float));
    cudaMalloc((void**)&d_B, (N - 2) * sizeof(float));

    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    int numBlocks = (N - 2 + BLOCK_SIZE - 1) / BLOCK_SIZE;
    izracunajB<<<numBlocks, BLOCK_SIZE>>>(d_A, d_B);

    cudaMemcpy(B, d_B, (N - 2) * sizeof(float), cudaMemcpyDeviceToHost);

    printf("B = ");
    for (int i = 0; i < N - 2; i++)
    {
        printf("%f ", B[i]);
    }
    printf("\n");
    cudaFree(d_A);
    cudaFree(d_B);

    return 0;
}

B = 1.000000 2.666667 5.000000 8.000000 11.666667 16.000000 21.000000 26.666666 33.000000 40.000000 47.666668 56.000000 65.000000 74.666664 85.000000 96.000000 107.666664 120.000000 133.000000 146.666672 161.000000 176.000000 191.666672 208.000000 225.000000 



In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5sftfuj0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5sftfuj0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=1e6dd26651778e91ba37a27729192e0bde7f3a4f3cd21f65c383cba1c7bb4d7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n4ryhqmc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
